### TruLens explainability

This is a notebook based on the tutorial from the original [trulens-repo](https://github.com/truera/trulens) with a patient safety incident report set of examples - all entirely fictitious, and not neccessarily accurate. The main purpose of this notebook is to provide a quick guide to using `trulens`.

This repo has a couple of different ways to train and setup transformer based classification models - one using the Transformers AutoModelForSequenceClassification classes, and the other with a customised classification heads etc.

Annoyingly this requires some careful consideration when taking these models and loading them into other frameworks. 

In [ ]:
import sys

import torch
from IPython.display import display
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from trulens.utils.nlp import token_baseline

# add the sys path for models
sys.path.append("../")

from models.transformer_plms.hf_transformer_classifier import IncidentModel
from trulens.nn.attribution import Cut, IntegratedGradients, OutputCut

# Now wrap in TruLens classes
from trulens.nn.models import get_model_wrapper
from trulens.nn.quantities import ClassQoI
from trulens.utils.typing import ModelInputs
from trulens.visualizations import NLP

In [ ]:
# set directory for any custom models
model_dir = "./models/"

In [ ]:
# set up some arguments dependent on the format of the model being loaded in
if "autoforsequence" in model_dir:
    model_type = "autoforsequence"
else:
    model_type = "customclassifier"

if "pretrained_format" in model_dir:
    pretrained_format = True

In [ ]:
print(model_type, pretrained_format)

In [ ]:
# Wrap all of the necessary components.
class IncidentSeverity:

    device = "cpu"
    # Can also use cuda if available:
    # device = 'cuda:0'

    if model_type == "autoforsequence":
        if pretrained_format:
            model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(
                device
            )
            tokenizer = AutoTokenizer.from_pretrained(model_dir)
        else:
            # load from IncidentModel class instead
            model = IncidentModel.load_from_checkpoint(
                f"{model_dir}/best-checkpoint.ckpt", model_type=model_type
            )
    else:
        model = IncidentModel.load_from_checkpoint(
            f"{model_dir}/best-checkpoint.ckpt", model_type=model_type
        )

        # tokenizer name
        tokenizer_model_name = model.model.config.name_or_path

        tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)

    labels = ["low", "high"]

    NEGATIVE = labels.index("low")

    POSITIVE = labels.index("high")


task = IncidentSeverity()

In [ ]:
sentences = [
    "patient was left waiting with a very high blood pressure for longer than advised. Patient seemed very agitated by the experience",
    "Patient was left waiting for 10 minutes",
    "Nothing out of the ordinary",
]

# Input sentences need to be tokenized first.

inputs = task.tokenizer(sentences, padding=True, return_tensors="pt").to(task.device)
# pt refers to pytorch tensor

# The tokenizer gives us vocabulary indexes for each input token (in this case,
# words and some word parts like the "'m" part of "I'm" are tokens).

print(inputs)

# Decode helps inspecting the tokenization produced:

print(task.tokenizer.batch_decode(torch.flatten(inputs["input_ids"])))
# Normally decode would give us a single string for each sentence but we would
# not be able to see some of the non-word tokens there. Flattening first gives
# us a string for each input_id.

In [ ]:
outputs = task.model(**inputs)

print(outputs)

# From logits we can extract the most likely class for each sentence and its readable label.

predictions = [task.labels[i] for i in outputs.logits.argmax(axis=1)]

for sentence, logits, prediction in zip(sentences, outputs.logits, predictions):
    print(logits.to("cpu").detach().numpy(), prediction, sentence)

In [ ]:
task.wrapper = get_model_wrapper(task.model, device=task.device)

# Attributions

Applying integrated gradents to the sentiment model is similar as in the prior notebooks except special considerations need to be made for the cuts used as the targets of the attribution (i.e. what do we want to assign importance to). As you may have noted above, the model takes as input integer indexes associated with tokens. As we cannot take gradient with respect to these, we use an alternative: the embedding representation of those same inputs. To instantiate trulens with this regard, we need to find inspect the layer names inside our model:

In [ ]:
task.wrapper.print_layer_names()

## Parameters

Above, `roberta_embeddings_word_embeddings` or `model_embeddings_word_embeddings` is the layer that produces a continuous representation of each input token so we will use that layer as the one defining the **distribution of interest**. While most neural NLP models contain a token embedding, the layer name will differ.

The second thing to note is the form of model outputs. Specifically, outputs are structures which contain a 'logits' attribute that stores the model scores.

Putting these things together, we instantiate `IntegratedGradients` to attribute each embedding dimension to the maximum class (i.e. the predicted class).

In [ ]:
embeds_layer_name = "roberta_embeddings_word_embeddings"
infl_max = IntegratedGradients(
    model=task.wrapper,
    doi_cut=Cut(f"{embeds_layer_name}"),
    qoi_cut=OutputCut(accessor=lambda o: o["logits"]),
)

In [ ]:
# Alternatively we can look at a particular class:

infl_positive = IntegratedGradients(
    model=task.wrapper,
    doi_cut=Cut(f"{embeds_layer_name}"),
    qoi=ClassQoI(task.POSITIVE),
    qoi_cut=OutputCut(accessor=lambda o: o["logits"]),
)

Getting attributions uses the same call as model evaluation.

In [ ]:
attrs = infl_max.attributions(**inputs)

for token_ids, token_attr in zip(inputs["input_ids"], attrs):
    for token_id, token_attr in zip(token_ids, token_attr):
        # Not that each `word_attr` has a magnitude for each of the embedding
        # dimensions, of which there are many. We aggregate them for easier
        # interpretation and display.
        attr = token_attr.sum()

        word = task.tokenizer.decode(token_id)

        print(f"{word}({attr:0.3f})", end=" ")

    print()

A listing as above is not very readable so Trulens comes with some utilities to present token influences a bit more concisely. First we need to set up a few parameters to make use of it:

In [ ]:
V = NLP(
    wrapper=task.wrapper,
    labels=task.labels,
    decode=lambda x: task.tokenizer.decode(x),
    tokenize=lambda sentences: ModelInputs(
        kwargs=task.tokenizer(sentences, padding=True, return_tensors="pt")
    ).map(lambda t: t.to(task.device)),
    # huggingface models can take as input the keyword args as per produced by their tokenizers.
    input_accessor=lambda x: x.kwargs["input_ids"],
    # for huggingface models, input/token ids are under input_ids key in the input dictionary
    output_accessor=lambda x: x["logits"],
    # and logits under 'logits' key in the output dictionary
    hidden_tokens=set([task.tokenizer.pad_token_id])
    # do not display these tokens
)

print("QOI = MAX PREDICTION")
display(V.token_attribution(sentences, infl_max))

print("QOI = POSITIVE")
display(V.token_attribution(sentences, infl_positive))

# Baselines

We see in the above results that special tokens such as the sentence end **&lt;/s&gt;** contributes are found to contribute a lot to the model outputs. While this may be useful in some contexts, we are more interested in the contributions of the actual words in these sentences. To focus on the words more, we need to adjust the **baseline** used in the integrated gradients computation. By default in the instantiation so far, the baseline for each token is a zero vector of the same shape as its embedding. By making the basaeline be identicaly to the explained instances on special tokens, we can rid their impact from our measurement. Trulens provides a utility for this purpose in terms of `token_baseline` which constructs for you the methods to compute the appropriate baseline. 

In [ ]:
# this needs to called differently based on whether it is using the AutoModel.from_pretrained class or not
if pretrained_format:
    inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
        keep_tokens=set([task.tokenizer.cls_token_id, task.tokenizer.sep_token_id]),
        # Which tokens to preserve.
        replacement_token=task.tokenizer.pad_token_id,
        # What to replace tokens with.
        input_accessor=lambda x: x.kwargs["input_ids"],
        # this model.model.get_input... is for InstanceModel classes as the PLM component is inside the a sub-class named model...
        ids_to_embeddings=task.model.get_input_embeddings()
        # Callable to produce embeddings from token ids.
    )
else:

    inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
        keep_tokens=set([task.tokenizer.cls_token_id, task.tokenizer.sep_token_id]),
        # Which tokens to preserve.
        replacement_token=task.tokenizer.pad_token_id,
        # What to replace tokens with.
        input_accessor=lambda x: x.kwargs["input_ids"],
        # this model.model.get_input... is for InstanceModel classes as the PLM component is inside the a sub-class named model...
        ids_to_embeddings=task.model.model.get_input_embeddings()
        # Callable to produce embeddings from token ids.
    )

We can now inspect the baselines on some example sentences. The first method returned by `token_baseline` gives us token ids to inspect while the second gives us the embeddings of the baseline which we will pass to the attributions method.

In [ ]:
print("originals=", task.tokenizer.batch_decode(inputs["input_ids"]))

baseline_word_ids = inputs_baseline_ids(
    model_inputs=ModelInputs(args=[], kwargs=inputs)
)
print("baselines=", task.tokenizer.batch_decode(baseline_word_ids))

In [ ]:
infl_max_baseline = IntegratedGradients(
    model=task.wrapper,
    resolution=50,
    baseline=inputs_baseline_embeddings,
    doi_cut=Cut(f"{embeds_layer_name}"),
    qoi_cut=OutputCut(accessor=lambda o: o["logits"]),
)


infl_positive_baseline = IntegratedGradients(
    model=task.wrapper,
    resolution=50,
    baseline=inputs_baseline_embeddings,
    doi_cut=Cut(f"{embeds_layer_name}"),
    qoi=ClassQoI(task.POSITIVE),
    qoi_cut=OutputCut(accessor=lambda o: o["logits"]),
)


print("QOI = MAX PREDICTION")
display(V.token_attribution(sentences, infl_max_baseline))

print("QOI = POSITIVE WITH BASELINE")
display(V.token_attribution(sentences, infl_positive_baseline))